## RAG: Retrieval Augmented Generation.
- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query -> Search -> Results -> (LLM) -> Answer

## Keyword VS Semantic Search 
![Vector](https://blog.dataiku.com/hs-fs/hubfs/dftt%202.webp?width=1346&height=632&name=dftt%202.webp)

from https://blog.dataiku.com/semantic-search-an-overlooked-nlp-superpower

![Emb_search](figures/emb_search.png)

from https://sreent.medium.com/llms-embeddings-and-vector-search-d4bd9362df56

In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25
! pip3 install -qU langchain_chroma

In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_upstage import UpstageEmbeddings

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large", embed_batch_size=100)
embeddings = embeddings_model.embed_documents(
    [
        "What is the best season to visit Korea?",
    ])

len(embeddings), len(embeddings[0])

(1, 4096)

In [5]:
# RAG 1. load doc (done), 2. chunking, splits, 3. embeding - indexing, 4. retrieve


In [6]:
from langchain_core.documents import Document

In [15]:
import json
question_and_answer_list = []
with open('json/question_and_answer_data.json', encoding='utf-8') as f:
    a = json.load(f)

In [16]:
for k, v in a.items():
    seq = ''
    for i in v:
        seq += f' {i}'
    question_and_answer_list.append(Document(page_content=seq, metadata={"test": "test"}))    

In [17]:
from langchain_chroma import Chroma

# 3. Embed & indexing
vectorstore = Chroma.from_documents(documents=question_and_answer_list, embedding=UpstageEmbeddings(model="solar-embedding-1-large", embed_batch_size=100))

챗봇 프롬프트 입력 

In [34]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage(temperature=0.8)

In [45]:
# 4. retrive
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("전공은 어떻게 선택?")
#result_docs = retriever.invoke("{message}")
print(len(result_docs))
print(result_docs)
#print(result_docs[0].page_content[:100])

4
[Document(page_content=' Answer:  혹시 장래 희망 정한 게 있어요? Question:  아니요. Answer:  그럼 부모님이 너한테는 이 직업이 어울린다고 추천해준 직업은요? Question:  그냥 했으면 좋겠다고 말한 적은 있는데, 어울린다고 말한 적은 없는 것 같아요. Answer:  했으면 좋겠다고 한 직업은 뭘까요? Question:  한의사요. Answer:  학생도 관심 있는 직업이에요? Question:  관심이 있긴 한데 그렇게 많이 관심 있진 않아요. Answer:  선생님 부모님도 선생님 어렸을 때 한의사 되었으면 좋겠다고 하셨었어요 Question:  그렇구나 Answer:  공부를 잘해야 할 수 있는 직업이죠 Answer:  ENFP 유형에게 맞는 직업은 아래와 같습니다. 1. 크리에이티브– 크리에이티브 디렉터, 디자이너, 시나리오 작가, 싱어송라이터, 방송 프로듀서, 연예인 2. 기획, 마케팅– 홍보·마케팅 컨설턴트, 카피라이터, 아트 디렉터 3. 교육, 상담– 각종 상담사, 주택 관리사, 교육 강사, 레크리에이션 강사, 예체능 교사 4. 비즈니스-컨설턴트, 이벤트, 엔터테인먼트, 마케팅, 헤드헌터, 상품기획, 코칭 성격에 어울리는 직업들이에요 혹시 관심 있는 직업 있으면 이야기해보세요 Question:  엄마가 전에 교사 되면 잘하겠다는 말 한 적이 있어요. 학교에서 진로 조사서 쓸 때 선생님이 꼭 쓰라 해서 카피라이터 조사한 적이 있어요. Answer:  어떤 선생님에 관심이 있나요? Question:  잘 모르겠어요. Answer:  역사 좋아하면 역사 선생님도 좋겠는데요? 선생님도 직업이 선생님이지만 너무 만족하고 있답니다 다시 태어나도 선생님이 하고 싶은 정도로요 Question:  많이 힘들 것 같아요. Answer:  그래도 교사는 힘든 직업은 아니에요 상대적으로 스트레스가 덜한 직업이랍니다 Question:  근데 딱히 하고싶다는 생각을 한 적이 없어요. Answer:  그래요 아직 초등학

In [53]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage(temperature=0.5)

prompt_template = PromptTemplate.from_template(
    """
    This is the Career Counseling Guidelines
    1. Familiarize yourself with the basic history of students' individual career paths.
    2. Determine the level and type of assistance based on students' career issues.
    3. Provide students with useful career-related information and resources.
    4. Share specific examples or experiences related to career paths honestly.
    5. Tell or connect students with specific examples from their seniors' career paths.
    6. Provide detailed guidance on career exploration possibilities related to their major.
    This is the list you should not do. 
    1. Do not convey stereotypes or biases about specific careers or jobs.
    2. Do not assume that your own success experiences apply to all students.
    3. Be careful not to unilaterally guide students in the direction you want.
    4. Do not do other work during the counseling session.
    5. Do not view career counseling as a performance metric.
    6. Avoid giving advice unrelated to the students' help requests.
    7. Be cautious of providing outdated information or advice about recent job market trends and conditions.
    8. Avoid giving unilateral instructions or reprimands.
    9. Do not ask many personal questions unrelated to career counseling.
    10. Avoid saying things that might hurt or lower students' morale.
    
    Follow above rules, and find similar question in the Context and give the answer similar to corresponding Answer. 
    Suppose you are a very good student counseler. Speak like you are a good counseler.

    This is example conversation.
    "Answer: Do you have any plans for this vacation?"
    "Question: Nothing much, just exercising at home."
    "Answer: Then, before our next session, how about researching one or more things you want to do? You can use the site I mentioned or search freely."
    "Question: Sounds good."
    "Answer: The second semester of your second year will be quite an important time. It would be good to get to know yourself better. I see a cat picture in your profile. Do you like animals? Jobs related to pets are also quite promising."
    "Question: I like cats."

    You should speak the "Answer" Part. 


    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {message}
    ---
    Context: {Context}
   
    """
)
chain = prompt_template | llm | StrOutputParser()

In [54]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    return chain.invoke({"message": message, "history": history_langchain_format})

In [55]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "전공 선택은 어떻게 하는 것이 좋을까 ? ",
            "유망 직업은 뭘까?",
            "How to make a chatbot?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [46]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [56]:
chain.invoke({"question":"저는 과학이 좋아요","Context":result_docs})

'답변: 과학을 좋아하신다면, 과학 관련 학과나 직업에 대해 더 알아보는 것은 어떨까요? 과학 연구원, 과학 교사, 과학 작가 등 다양한 직업이 있습니다.'

In [ ]:
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [41]:
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 528, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\blocks.py", line 1908, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFounda

In [20]:
chain.invoke({"question": "저는 게임을 좋아해요.", "Context": result_docs})

'학생님께서 게임을 좋아하시는군요. 게임을 좋아하고 잘하시는 것은 멋진 일입니다. 게임을 취미로 즐기는 것도 좋지만, 프로게이머로 진로를 생각하고 계시다면, 프로게이머가 되기 위해 필요한 기술과 지식을 습득하는 것이 중요합니다.\n\n프로게이머가 되기 위해서는 게임 실력과 함께 전략적인 사고와 빠른 판단력, 그리고 팀워크 등이 필요합니다. 또한, 게임에 대한 지식과 최신 트렌드를 파악하는 것도 중요합니다.\n\n프로게이머로 진로를 선택하신다면, 게임 대회에 참가하거나 게임 관련 커뮤니티에 참여하여 경험을 쌓는 것도 좋은 방법입니다. 또한, 게임 관련 교육 기관이나 학원을 찾아보는 것도 도움이 될 수 있습니다.\n\n게임을 좋아하고 잘하시는 학생님께서는 프로게이머로 진로를 선택하신다면, 많은 가능성을 가지고 계실 것입니다. 하지만, 게임 외에도 다양한 진로가 있으니, 다른 분야에도 관심을 가지고 탐색해보시는 것도 좋은 방법입니다.\n\n학생님의 진로를 응원하며, 언제든지 도움이 필요하시면 말씀해주세요.'

# Excercise: Hybrid
Sometimes keyword search can be useful. Design a system that does keyword and semantic search, then combine the results. Use them as context for your RAG.